In [9]:
from transaction_analysis.profit_scanner import ProfitScanner
from utils.transaction.swap_transaction_analyzer import *

from utils.transaction.erc20_transaction import ERC20Transaction
from utils.transaction.normal_transaction import NormalTransaction
from utils.transaction.internal_transaction import InternalTransaction

In [10]:
#enable autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
owner_address = "0x1a869357c99bdf201D0378971FfE0b78DA5697E0"

profit_scanner = ProfitScanner(owner_address)

In [12]:
txs_by_hash = profit_scanner.create_txs_by_hash()
function_name_by_hash = profit_scanner.create_function_name_by_hash()

In [13]:
tx_hash = "0x8f8728f9a1b68cfec612bbd4224da1e08a2b8ff10f3473af5860a952c65d54f4"

def get_bundle_by_tx_hash(tx_hash):
    txs = txs_by_hash[tx_hash]

    normal_transaction, erc20_transactions, internal_transaction = None, [], None

    for tx in txs:
        if isinstance(tx, NormalTransaction):
            normal_transaction = tx
        elif isinstance(tx, ERC20Transaction):
            erc20_transactions.append(tx)
        elif isinstance(tx, InternalTransaction):
            internal_transaction = tx
        else:
            raise Exception("Unknown transaction type", type(tx))

    return normal_transaction, erc20_transactions, internal_transaction

normal_transaction, erc20_transactions, internal_transaction = get_bundle_by_tx_hash(tx_hash)

In [14]:
data = []

for tx_hash in txs_by_hash:
    if tx_hash not in function_name_by_hash:
        continue
    if is_swap(function_name_by_hash[tx_hash]):
        normal_transaction, erc20_transactions, internal_transaction = get_bundle_by_tx_hash(tx_hash)
        try:
            swap_transaction_analyzer = SwapTransactionAnalyzer(owner_address, normal_transaction, erc20_transactions, internal_transaction)
            data.append(swap_transaction_analyzer.get_swap_transaction_result().to_json())
        except IndexError as e:
            pass
            print(e)
            print(tx_hash)
            print(normal_transaction)
            print(erc20_transactions)
            print(internal_transaction)

In [15]:
import pandas as pd

df = pd.DataFrame(data)

In [16]:
df.function_name.value_counts()

function_name
swapExactTokensForETHSupportingFeeOnTransferTokens(uint256 amountIn, uint256 amountOutMin, address[] path, address to, uint256 deadline)    82
Name: count, dtype: int64